### ![](../../img/installation-ico.png) Installation 

In [ ]:
pip install langchain langchain-google-vertexai google.auth

### ![](../../img/package-ico.png) Imports

In [ ]:
import os
import google.auth

from langchain_google_vertexai import VertexAI

# pour le template du prompt d'entrée
from langchain_core.prompts import PromptTemplate

# pour le parser de sortie
from langchain_core.output_parsers import StrOutputParser

### ![](../../img/parametrage-ico.png) Paramétrages 

In [ ]:
# Gestion credentials VertexAI

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../settings/credentials.json"
google.auth.default()

In [ ]:
# notre modèle
llm = VertexAI(model_name="gemini-1.0-pro", temperature=0)

# notre prompt de chat
prompt = PromptTemplate.from_template("Tell me a joke about {input}.")
# notre parser de sortie
output_parser = StrOutputParser()

# notre chaîne d'exécution
chain = prompt | llm | output_parser

### ![](../../img/jouer-ico.png) Exécution

In [ ]:
# invocation simple
input = "clown"
chain.invoke({"input": input})

## Aller plus loin

### ![](../../img/jouer-ico.png) Stream

In [ ]:
# invocation pour stream
for chunk in chain.stream({"input": input}):
    print(chunk, end="", flush=True)

### ![](../../img/jouer-ico.png) Batch

In [ ]:
chain.batch([
    {"input": "developers"},
    {"input": "magician"}
])

### ![](../../img/jouer-ico.png) ChatPrompt et JsonParser

In [ ]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_google_vertexai import ChatVertexAI
from pprint import pprint

# Le LLM
llm = ChatVertexAI(model_name="gemini-1.0-pro", temperature=0)

# Le schéma de données attendu en sortie
class Response(BaseModel):
    answer: str = Field(description="answer to the user's question")
    source: str = Field(description="source used to answer the user's question, should be a website")

# Le parser avec le schéma
json_parser = JsonOutputParser(pydantic_object=Response)

format_instructions = json_parser.get_format_instructions()

# Le ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages(
    [
        # Message système pour spécifier le comportement ainsi que le format de sortie
        SystemMessage(content=(f"""Answer the users question as best as possible.
        {format_instructions}
        """)),
        
        # Message humain à partir d'un template
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

# Chaînage
chain_chat = chat_prompt | llm | json_parser

question = "What is the capital of Norway?"

# FACTULTATIF : Affichage du prompt
pprint(chat_prompt.format_messages(question=question))

chain_chat.invoke({"question": question})